In [ ]:
import torch
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np

# Exercise 1. Total variation

In parametric statistics we are interested in estimating parameters of the distribution. In order to do that, we need to define an appropriate distance metric between the distributions. Such a common metric in classical statistics is total variation, which is defined as $TV(P, Q) = \max_{A \in E}{|P(A) - Q(A)|}$, where $E$ is the union of domains of $P$ and $Q$.  Total variation is a proper distance, meaning that it satisfies the following properties:
$$
TV(P, Q) = TV(Q,P) \ (\text{symmetric}) \\
TV(P, Q) \geq 0 \ (\text{positive}) \\
\text{if} \ TV(P, Q) = 0, \ \text{then} \  P = Q \ (\text{definite}) \\
TV(P, Q) \leq TV(P, K) + TV(K, Q) \ (\text{triangle inequality})
$$

$\textbf{Your task is to prove these properties} \\ $
Write your solutions in LateX or attach a picture in the answer cell provided below. You can add a picture using the command ```![1](imagename_in_the_folder.jpg)```. Latex in here works similarly as you would write it normally!

YOUR ANSWER HERE

When both $P$ and $Q$ are either discrete or continuous, TV can be computed as $\frac{1}{2}{\sum_{x \in E}}{|P(x) - Q(x)|}$ or $\frac{1}{2}{\int_{x \in E}}{|P(x) - Q(x)| dx}$. $ \\ $ However, despite the intuative definition, TV has a major disadvantage - we cannot easily construct an estmator out of it, when we have access only to samples of target distribution $P$ and our goal is to infer parameters of $Q$. Hence, we need to define more appealing metric.

# Exercise 2. KL divergence

KL divergence is, probably, the most popular statistical metric for measuring how $P$ is close to $Q$. Mathematically KL is not a distance, in general it does not satisfy properties of symmetry and triangle inequality, however, it does not really affect the applicability of this metric. KL is still positive and definite and positiviness is proved by employing Jensen's inequality. This inequality appears a lot when deriving lower bounds for latent variable models (next lectures), so lets focus on it a little bit.

### Jensen’s inequality

if $X$ is a random variable and $f$ is a convex function, then
$$
f(\mathbb{E}[f(X)]) \leq \mathbb{E}[f(X)]
$$

A differentiable real-valued function f of a single variable is called convex if the first order Taylor approximation of f is an underestimate of the function, that is
$$
f(c) + (x-c) f'(c) \leq f(x)
$$
for every $c$ and $x$.

This definition of the convexity differs from the canonical one, however is more useful for proving inequality. $\textbf{Your task is to prove Jensen’s inequality}$. (Hint: choose $c$ as $\mathbb{E}[X])$. Write your solutions in LateX or attach a picture in the answer cell provided below. You can add a picture using the command ```![1](imagename_in_the_folder.jpg)```. Latex in here works similarly as you would write it normally!

YOUR ANSWER HERE

To start exploring KL, we will consider a situation, where we want to minimise KL between two distributions: the target distribution $p$ is known to us and for the approximation distribution $q$ we know its parametric form. For simplicity, we will assume that both distributions belong to the Gaussian family. $\textbf{Your task is to optimise parameters of $q$ using the gradient descent}$. Below N(a, b) corresonds to normal distribution with mean $a$ and variance $b$.

In [ ]:
def KL(mean, log_var, mean_target, log_var_target):
    """
    Args:
        mean: mean of normal distribution q which we optimise
        log_var: log of variance of normal distribution q which we optimise
        mean_target: mean of target normal distribution p which is known
        log_var_target: log of variance of target normal distribution p which is known
        
    Returns:
        KL between p and q
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return kl
    

In [ ]:
res = KL(torch.tensor(0),torch.tensor(1).log(),torch.tensor(0),torch.tensor(3).log()).item()
eps = 0.001

diff = abs(res - 0.4506938)
assert diff < eps, \
print(f"KL between N(0,1) and N(0, 3) do not allign, the absolute value difference is {diff}")
print("Success")

In [ ]:
# Here is a hidden test


In [ ]:
mean = torch.nn.Parameter(torch.randn(1))
log_var = torch.nn.Parameter(torch.randn(1))

mean_target = torch.tensor(5)
log_var_target = torch.log(torch.tensor(2.0))

num_iterations = 5000
optimiser = torch.optim.Adam([mean, log_var], lr=1e-2)

In [ ]:
for i in range(num_iterations):
    optimiser.zero_grad()
    loss = KL(mean, log_var, mean_target, log_var_target)
    if i % 100 == 0:
        print('Iter %d/%d - KL Loss: %.3f' % (i, num_iterations, loss.item()), flush=True)
    loss.backward()
    optimiser.step()

print("optimised mean: ", mean.item())

print("optimised variance: ", log_var.exp().item())

In [ ]:
eps = 0.1
assert abs(mean.detach().item() - mean_target) < eps, \
print(f"Means do not allign, the absolute value difference is {abs(mean.detach().item() - mean_target)}")
assert abs(log_var.detach().item() - log_var_target) < eps,\
print(f"log variances do not allign, the absolute value difference is {abs(log_var.detach().item() - log_var_target)}")

print("Success")

Lets draw the resulting densities

In [ ]:
x = np.linspace(0,10,1000)
target_pdf = scipy.stats.norm(mean_target, log_var_target.exp().sqrt().numpy()).pdf(x)
fit_pdf = scipy.stats.norm(mean.detach().numpy(), log_var.exp().sqrt().detach().numpy()).pdf(x)

plt.plot(x, target_pdf, label="target")
plt.plot(x, fit_pdf, label="fit")
plt.legend()
plt.show()

In most real cases, of course, we are not provided with the target distribution, but rather can observe samples from it. However, we can still define a parametric form for $q$ and minimise KL between emperical distribution of the data and $q$. Assume that $q$ belongs to the Gaussian family again. $\textbf{Your task is to optimise parameters of $q$ using the gradient descent}$.

In [ ]:
def loss_function(mean, log_var, samples, N):
    """
    Args:
        mean: mean of normal distribution q which we optimise
        log_var: log of variance of normal distribution q which we optimise
        samples: samples from the unknown distribution p
        N: number of samples
        
    Returns:
        objective for minimising KL
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return loss

In [ ]:
mean = torch.nn.Parameter(torch.randn(1))
log_var = torch.nn.Parameter(torch.randn(1))
optimiser = torch.optim.Adam([mean, log_var], lr=1e-2)
N = 50000
samples = torch.randn(N) * torch.sqrt(log_var_target.exp()) + mean_target

In [ ]:
for i in range(num_iterations):
    optimiser.zero_grad()
    loss = loss_function(mean, log_var, samples, N)
    if i % 100 == 0:
        print('Iter %d/%d - KL Loss: %.3f' % (i, num_iterations, loss.item()), flush=True)
    loss.backward()
    optimiser.step()

print("optimised mean: ", mean.item())

print("optimised variance: ", log_var.exp().item())

In [ ]:
eps = 0.1
assert abs(mean.detach().item() - mean_target) < eps, \
print(f"Means do not allign, the absolute value difference is {abs(mean.detach().item() - mean_target)}")
assert abs(log_var.detach().item() - log_var_target) < eps,\
print(f"log variances do not allign, the absolute value difference is {abs(log_var.detach().item() - log_var_target)}")

print("Success")

Lets draw the resulting densities

In [ ]:
target_pdf = scipy.stats.norm(mean_target, log_var_target.exp().sqrt().numpy()).pdf(x)
fit_pdf = scipy.stats.norm(mean.detach().numpy(), log_var.exp().sqrt().detach().numpy()).pdf(x)

plt.plot(x, target_pdf, label="target")
plt.plot(x, fit_pdf, label="fit")
plt.legend()
plt.show()

# Exercise 3. Jensen-Shannon divergence

KL is not the only divergence for estimating difference between probability distributions. Another example is Jensen-Shannon divergence (JSD), which is defined as
$$
\text{JS}(P, Q) = \frac{1}{2}\text{KL}(P, M) + \frac{1}{2}\text{KL}(Q, M),
$$
where $M = \frac{1}{2}(P + Q)$. Despite being symmetric, even for gaussians it is analytically intractable, hence numerical solutions are used. One option is to estimate corresponding KL terms by sampling from P and Q. $\textbf{Your task is to compute JSD between two gaussians by employing sampling}$. You can use scipy.stats package here.

In [ ]:
def JSD(mean1, sigma1, mean2, sigma2, N):
    """
    Args:
        mean1: mean of normal distribution p
        sigma1: std of the normal distribution p 
        mean2: mean of normal distribution q
        sigma2: std of the normal distribution q
        N: number of simulations
        
    Returns:
        jsd
    """
    samples1 = np.random.randn(N) * sigma1 + mean1
    samples2 = np.random.randn(N) * sigma2 + mean2
    # YOUR CODE HERE
    raise NotImplementedError()
    return jsd

In [ ]:
eps = 0.028
jsd = 0.545333

diff = abs(JSD(3,6, 10,0.5, 10000) - jsd)
assert diff < eps, \
print(f"jsd do not allign, the absolute value difference is {diff}")

print("Success")

In [ ]:
# Here is a hidden test


# Exercise 4. Monte-Carlo for area estimation

Monte-Carlo approaches are used for various tasks. One of them is area estimation, when the shape is complex. For this task, your are given figure that satisfies the following inequalities:

$$
\frac{x^2}{2} + \frac{y^2}{3} < 1 \\
x + y > -1 \\
x - y < 0
$$

$\textbf{Your task is to estimate its area}$. Sample $x$ and $y$ from uniform distributions over $[-2, 2]$ and compute the Monte-Carlo estimate.

In [ ]:
def area(N):
    """
    Args:
        N: number of samples from uniform distributions
    Returns:
        x: sampled x values
        y: sampled y values
        mask: boolean array of size N where mask[i] is True if corresponding x_i and y_i satisfy inequalities
        area: estimated area
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return x,y,mask,area

In [ ]:
# Here is a hidden test


In [ ]:
x,y,mask,area_ = area(1000)

plt.scatter(x,y, label="all points")
plt.scatter(x[mask], y[mask], label="points in the region")
plt.legend()
plt.show()